In [ ]:
import tensorflow as tf
from skimage import io
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.models import Model
import os
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

import cv2 as cv
from skimage import feature as skif
from google.colab import drive
drive.mount('/content/drive')
import random

Mounted at /content/drive


In [ ]:
print(tf.__version__)

2.4.1


In [ ]:
model_loaded = tf.keras.models.load_model('/content/drive/MyDrive/Dataset/anti_spoofing/feature_extraction.h5')

In [ ]:
model_loaded.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 5, 5, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_7 (Dense)              (None, 1)                

In [ ]:
# datagen = ImageDataGenerator(rescale=1./255)
datagen = ImageDataGenerator()
batch_size = 32
img_width = 224
img_height = 224

layer_name = 'dense_6'
intermediate_layer_model = Model(inputs=model_loaded.input, outputs=model_loaded.get_layer(layer_name).output)

In [ ]:
# Calcul de l'histograme LBP

def lbp_histogram(image, P=8, R=1, method='nri_uniform'):
    '''
    image: shape is N*M 
    '''
    lbp = skif.local_binary_pattern(image, P,R, method) # lbp.shape is equal image.shape
    # cv2.imwrite("lbp.png",lbp)
    # plt.imshow(lbp)
    max_bins = int(lbp.max() + 1) # max_bins is related P
    hist, _ = np.histogram(lbp,  density=True, bins=max_bins, range=(0, max_bins))
    return lbp, hist

In [ ]:
def extract_features(directory, intermediate_layer_model):
  person_content = ['live/', 'spoof/']
  label = {'live/': 0, 'spoof/': 1}
  liste_images_classe = []
  features = []
  labels = []

  for folder in person_content:
      list_person_folder = os.listdir(directory+folder)
      for im_name in list_person_folder:
        if im_name.split('.')[1] in ['png', 'jpg']: 
          imag = [os.path.join(directory+folder, im_name), label[folder]]
          liste_images_classe.append(imag)

  random.shuffle(liste_images_classe)

  # Pass data through convolutional base
  with tqdm(total=len(liste_images_classe), desc="Chargement en cour ...", bar_format="{l_bar}{bar} [ time left: {remaining} ]") as pbar:
    for one_image in liste_images_classe:
      picture = cv.imread(one_image[0])

      # Extraction de LBP
      image_y = cv.cvtColor(picture, cv.COLOR_BGR2YCrCb)
      lbp_y, y_h = lbp_histogram(image_y[:,:,0]) # y channel
      lbp_cb, cb_h = lbp_histogram(image_y[:,:,1]) # cb channel
      lbp_cr, cr_h = lbp_histogram(image_y[:,:,2]) # cr channel
      feature_lbp = np.concatenate((y_h, cb_h, cr_h))

      # Extraction de caractéristiques de tensorflow
      picture = cv.cvtColor(picture, cv.COLOR_BGR2RGB)
      img = tf.image.resize(picture, [224, 224], method='nearest')
      X = image.img_to_array(img)
      X = np.expand_dims(X, axis=0)
      images = np.vstack([X])
      images = images / 255
      feature_tensorflow = intermediate_layer_model.predict([images])[0]

      if len(feature_lbp) == 177:
          feature_final = np.concatenate((feature_lbp, feature_tensorflow), axis=None)
          features.append(feature_final)
          labels.append(one_image[1])
      pbar.update(1)

  return features, labels

In [ ]:
train_dir = "/content/drive/MyDrive/Dataset/anti_spoofing/train/"
train_features, train_labels = extract_features(train_dir, intermediate_layer_model)

Chargement en cour ...: 100%|██████████ [ time left: 00:00 ]


In [ ]:
np.save("/content/drive/MyDrive/Dataset/anti_spoofing/train/features_all_train.npy", np.array(train_features))
np.save("/content/drive/MyDrive/Dataset/anti_spoofing/train/labels_all_train.npy", np.array(train_labels))

In [ ]:
test_dir = "/content/drive/MyDrive/Dataset/anti_spoofing/test/"
test_features, test_labels = extract_features(test_dir, intermediate_layer_model)

Chargement en cour ...: 100%|██████████ [ time left: 00:00 ]


In [ ]:
np.save("/content/drive/MyDrive/Dataset/anti_spoofing/test/features_all_test.npy", test_features)
np.save("/content/drive/MyDrive/Dataset/anti_spoofing/test/labels_all_test.npy", test_labels)